# Making a Reddit App for Authorization

1. Make a Reddit account and log in
2. Go to https://www.reddit.com/prefs/apps/
3. Create an App
4. Fill out the create application form 
  1. Choose the "script" option
  2. For our class, a redirect uri of http://soic.indiana.edu will suffice
5. After you've created the app, you'll see a window with your app's settings
  1. Get the client id - it's under your app's name
  2. Get the client secret
  
# Creating PRAW Reddit api object

In [ ]:
import praw

client_id = "h_emaHOS9whQlA" # insert your client ID here
client_secret = "wBdMS8TnWmB8wXNkw7JncZZJMO0" # client secret here
user_agent = "IU Social Media Mining by /r/vqmalic vmalic@indiana.edu" # a string identifying your app to agents; it is courteous practice to provide your contact info

r = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

## Getting a User and Their Submissions

In [ ]:
user = r.redditor("bboe") # bboe is a PRAW developer
print(user.name)

Get the user's 10 most recent submissions.

In [ ]:
stored_submissions = []
for submission in user.submissions.new(limit=10):
    stored_submissions.append(submission)

Submission objects have multiple useful properties.

submission = stored_submissions[1]

print("Submission author?")
print(submission.author)
print("How many upvotes?")
print(submission.ups)
print("And downvotes?")
print(submission.downs)
print("Total score (ups minus downs)")
print(submission.score)
print("If it is a text submission, what's the text?")
print(submission.selftext)

## Get a user's comments and scores

In [ ]:
stored_comments = []

for comment in user.comments.new(limit=10):
    stored_comments.append(comment)

In [ ]:
for comment in stored_comments:
    print(comment.body)
    print(comment.score)
    print("*"*50)

## Other ways of pulling submissions/comments

Note that we called the method ``new`` to get the user's latest comments. There are methods ``hot``, ``top``, and ``controversial`` that order your items differently. ``new`` does reverse chronological, ``top`` gets items with highest scores, ``controversial`` gets items that have similar levels of upvotes and downvotes, ``hot`` gets items that have gotten upvotes recently.

In [ ]:
# bboe's best submissions

for submission in user.submissions.top(limit=5):
    print(submission.title, submission.score)

# Getting Content from Subreddits

Let's make a Pythonic object representing the subreddit.

In [ ]:
subreddit_republican = r.subreddit("Republican")
subreddit_democrat = r.subreddit("democrats")

Get most recent comments and scores.

In [ ]:
comments_republican = []
comments_democrat = []

for c in subreddit_republican.comments(limit=10):
    comments_republican.append((c.body, c.score))
    
for c in subreddit_democrat.comments(limit=10):
    comments_democrat.append((c.body, c.score))

In [ ]:
print(comments_republican[0])
print("*"*50)
print(comments_democrat[0])

## Get Highest Rated Submissions from Last Week

In [ ]:
r_top_submissions = []
d_top_submissions = []

for s in subreddit_republican.top(limit=10, time_filter="week"):
    r_top_submissions.append((s.title, s.score))
    
for s in subreddit_democrat.top(limit=10, time_filter="week"):
    d_top_submissions.append((s.title, s.score))

In [ ]:
print(r_top_submissions[0])
print(d_top_submissions[0])

# Getting comments on submissions

Getting comments on submissions is a little complicated. With users or entire subreddits, it's simple, because there is a one-to-many correspondence between user/subreddit and the comments it has. However, comments on a submission are organized in a *tree-like structure*; that is, the submission itself may have comments, and those comments may have comments on them, and so on. Because of this, we don't have helpful organizing functions like ``new`` or ``top``. We have to get them all and organize them ourselves. 

First, let's get the most recent submission of a subreddit and try to get the *top-level comments* on that submission. 

In [ ]:
newssubreddit = r.subreddit("news")

submissions = []

for submission in newssubreddit.top(limit=5, time_filter="week"):
    submissions.append(submission)
    
s = submissions[0] # Let's work with the first submission

In [ ]:
comments = []

for top_level_comment in s.comments:
    comments.append(top_level_comment)

In [ ]:
# Text of the 11th comment
comments[0].body

Comments can have comments themselves. Here's how extract the children comments of the first comment on the original submission.

In [ ]:
replies = []

for reply in comments[0].replies:
    replies.append(reply)

In [ ]:
replies[0].body

PRAW deals with Reddit rate limitations on comments by inserting "MoreComments" objects into the comment tree. For example, at the time of me writing this code, the fourth item in replies is a "MoreComments" object.

In [ ]:
replies

We can open up a MoreComments object, but this necessitates sending another request to reddit.

In [ ]:
mc = replies[-1]

comments = []
for c in mc.comments():
    comments.append(c)

In [ ]:
comments[0].body

It's important to keep in mind if you work with reddit Comment Forests to calibrate your code to handle "MoreComments" objects gracefully. If you need help writing the code to do this, let me know.